In [1]:
import nibabel as nib
import pandas as pd
import os
import numpy as np
import os
import zipfile
import numpy as np
import tensorflow as tf
import random
import matplotlib.pyplot as plt

from scipy import ndimage
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.python.keras import layers
from scipy import ndimage


def read_nifti_image(file_path):
    """Read and load volume"""
    # Read file
    img = nib.load(file_path)
    # Get raw data
    img = img.get_fdata()
    return img


def normalize(volume):
    """Normalize the volume"""
    min = -1000
    max = 400
    volume[volume < min] = min
    volume[volume > max] = max
    volume = (volume - min) / (max - min)
    volume = volume.astype("float32")
    return volume


def resize_vol(img):
    """Resize across z-axis"""
    # Set the desired depth
    img_desired_depth = 64
    img_desired_width = 128
    img_desired_height = 128
    # Get current depth
    current_depth = img.shape[-1]
    current_width = img.shape[0]
    current_height = img.shape[1]
    # Compute depth factor
    depth = current_depth / img_desired_depth
    width = current_width / img_desired_width
    height = current_height / img_desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height
    # Rotate
    img = ndimage.rotate(img, 90, reshape=False)
    # Resize across z-axis
    img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1)
    return img


def scan_process(path):
    """Read and resize volume"""
    try:
        # Read scan
        volume = read_nifti_image(path)
        # Normalize
        volume = normalize(volume)
        # Resize width, height and depth
        volume = resize_vol(volume)
        return volume
    
    except Exception as e:
        print(path)
        print(repr(e))



In [33]:
labels_dataframe = pd.read_csv('/Users/kitap/dev/clemson/course_work/sem3/cpsc8650/BET_BSE_DATA/Label_file.csv')
labels_dataframe.head()

,Filename,Recognizable-Facial-Feature,Brain-Feature-Loss
0,IXI369-Guys-0924-T1_bet_03.nii,Yes,No
1,IXI448-HH-2393-T1_bet_07.nii,Yes,No
2,IXI252-HH-1693-T1_bet_08.nii,Yes,No
3,IXI188-Guys-0798-T1_bet_17.nii,Yes,No
4,IXI182-Guys-0792-T1_bet_17.nii,Yes,No


In [20]:
files_location = os.listdir('/Users/kitap/dev/clemson/course_work/sem3/cpsc8650/BET_BSE_DATA/files')
files_location

['IXI482-HH-2178-T1_bet_86.nii.gz',
 'IXI627-Guys-1103-T1_bet_83.nii.gz',
 'IXI538-HH-2411-T1_bse_high_s78_r2.nii.gz',
 'IXI436-HH-2153-T1_bse_less_s42_r1.nii.gz',
 'IXI622-Guys-1102-T1_bet_73.nii.gz',
 'IXI420-Guys-1028-T1_bet_85.nii.gz',
 'IXI515-HH-2377-T1_bet_03.nii.gz',
 'IXI460-Guys-0999-T1_bet_19.nii.gz',
 'IXI635-HH-2691-T1_bse_high_s84_r2.nii.gz',
 'IXI464-IOP-1029-T1_bet_08.nii.gz',
 'IXI431-Guys-0986-T1_bet_81.nii.gz',
 'IXI428-Guys-0996-T1_bse_high_s85_r2.nii.gz',
 'IXI330-Guys-0881-T1_bet_14.nii.gz',
 'IXI465-HH-2176-T1_bse_high_s89_r2.nii.gz',
 'IXI587-Guys-1128-T1_bet_07.nii.gz',
 'IXI389-Guys-0930-T1_bet_15.nii.gz',
 'IXI619-Guys-1099-T1_bse_high_s78_r2.nii.gz',
 'IXI341-Guys-0906-T1_bet_18.nii.gz',
 'IXI606-HH-2601-T1_bet_05.nii.gz',
 'IXI294-IOP-0868-T1_bse_high_s85_r2.nii.gz',
 'IXI311-Guys-0885-T1_bse_less_s45_r1.nii.gz',
 'IXI572-HH-2605-T1_bet_18.nii.gz',
 'IXI305-IOP-0871-T1_bet_87.nii.gz',
 'IXI541-IOP-1146-T1_bet_73.nii.gz',
 'IXI635-HH-2691-T1_bet_12.nii.gz',


In [48]:
labels_files = []
for k in files_location:
    labels_files.append(labels_dataframe[labels_dataframe['Filename'] == k.strip('.gz')]['Recognizable-Facial-Feature'])
labels_files[:5]

[581    No
 Name: Recognizable-Facial-Feature, dtype: object,
 443    No
 Name: Recognizable-Facial-Feature, dtype: object,
 1093    No
 Name: Recognizable-Facial-Feature, dtype: object,
 977    Yes
 Name: Recognizable-Facial-Feature, dtype: object,
 507    No
 Name: Recognizable-Facial-Feature, dtype: object]

In [50]:
normal_scan_paths = ['/Users/kitap/dev/clemson/course_work/sem3/cpsc8650/BET_BSE_DATA/files/' + l for l in files_location]
normal_scan_paths[:5]

['/Users/kitap/dev/clemson/course_work/sem3/cpsc8650/BET_BSE_DATA/files/IXI482-HH-2178-T1_bet_86.nii.gz',
 '/Users/kitap/dev/clemson/course_work/sem3/cpsc8650/BET_BSE_DATA/files/IXI627-Guys-1103-T1_bet_83.nii.gz',
 '/Users/kitap/dev/clemson/course_work/sem3/cpsc8650/BET_BSE_DATA/files/IXI538-HH-2411-T1_bse_high_s78_r2.nii.gz',
 '/Users/kitap/dev/clemson/course_work/sem3/cpsc8650/BET_BSE_DATA/files/IXI436-HH-2153-T1_bse_less_s42_r1.nii.gz',
 '/Users/kitap/dev/clemson/course_work/sem3/cpsc8650/BET_BSE_DATA/files/IXI622-Guys-1102-T1_bet_73.nii.gz']

In [6]:
len(normal_scan_paths)

803

In [51]:
normal_scans = np.array([scan_process(path) for path in normal_scan_paths])

/Users/kitap/dev/clemson/course_work/sem3/cpsc8650/BET_BSE_DATA/files/IXI340-IOP-0915-T1_bet_02.nii.gz
BadGzipFile('CRC check failed 0xf46a150d != 0x3e1973e5')


/var/folders/93/f_h08jrn2hs9z92c1rn61rn40000gq/T/ipykernel_6082/3894096321.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  normal_scans = np.array([scan_process(path) for path in normal_scan_paths])


In [82]:
labels = [1 if (label == 'Yes').bool() else 0 for label in labels_files]
labels[:5]

[0, 0, 0, 1, 0]

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(normal_scans, labels, test_size=0.33, random_state=42)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.3, random_state=42)

In [ ]:
len(X_train)

In [ ]:
%pip install tensorflow

In [ ]:
@tf.function
def rotate(volume):
    """Rotate the volume by a few degrees"""

    def scipy_rotate(volume):
        # define some rotation angles
        angles = [-20, -10, -5, 5, 10, 20]
        # pick angles at random
        angle = random.choice(angles)
        # rotate volume
        volume = ndimage.rotate(volume, angle, reshape=False)
        volume[volume < 0] = 0
        volume[volume > 1] = 1
        return volume

    aug_volume = tf.numpy_function(scipy_rotate, [volume], tf.float32)
    return aug_volume


def train_preprocessing(volume, label):
    """Process training data by rotating and adding a channel."""
    # Rotate volume
    volume = rotate(volume)
    volume = tf.expand_dims(volume, axis=3)
    return volume, label


def validation_preprocessing(volume, label):
    """Process validation data by only adding a channel."""
    volume = tf.expand_dims(volume, axis=3)
    return volume, label


In [ ]:
#enter the train test split cell here and rename the variable in train loader and validation loader
train_loader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
validation_loader = tf.data.Dataset.from_tensor_slices((x_val, y_val))

batch_size = 2
# Augment the on the fly during training.
train_dataset = (
    train_loader.shuffle(len(x_train))
    .map(train_preprocessing)
    .batch(batch_size)
    .prefetch(2)
)
# Only rescale.
validation_dataset = (
    validation_loader.shuffle(len(x_val))
    .map(validation_preprocessing)
    .batch(batch_size)
    .prefetch(2)
)

In [ ]:

def get_model(width=128, height=128, depth=64):
    """Build a 3D convolutional neural network model."""

    inputs = keras.Input((width, height, depth, 1))

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=256, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(units=512, activation="relu")(x)
    x = layers.Dropout(0.3)(x)

    outputs = layers.Dense(units=1, activation="sigmoid")(x)

    # Define the model.
    model = keras.Model(inputs, outputs, name="3dcnn")
    return model


# Build model.
model = get_model(width=128, height=128, depth=64)
model.summary()

In [ ]:
# Compile model.
initial_learning_rate = 0.0001
learning_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)
model.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=learning_schedule),
    metrics=["acc" , tf.keras.metrics.MeanSquaredError(),
        tf.keras.metrics.AUC(),tf.keras.metrics.Recall(),
        tf.keras.metrics.TruePositives(),
        tf.keras.metrics.FalseNegatives()],
)

# Define callbacks.
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    "3d_image_classification.h5", save_best_only=True
)
early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_acc", patience=15)

# Train the model, doing validation at the end of each epoch
epochs = 5
model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=epochs,
    shuffle=True,
    verbose=2,
    callbacks=[checkpoint_cb, early_stopping_cb],
)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 6))
ax = ax.ravel()

for i, metric in enumerate(["acc", "loss"]):
    ax[i].plot(model.history.history[metric])
    ax[i].plot(model.history.history["val_" + metric])
    ax[i].set_title("Model {}".format(metric))
    ax[i].set_xlabel("epochs")
    ax[i].set_ylabel(metric)
    ax[i].legend(["train", "validation"])

In [ ]:
model.load_weights("3d_image_classification.h5")
new_list=[]
for i in range(len(X_test)):
    prediction = model.predict(np.expand_dims(X_test[i], axis=0))[0] 
    scores = [1 - prediction[0], prediction[0]]
    new_list.append([prediction[0]])
new_list

In [ ]:
test_list=[]
for i in Y_test:
    test_list.append([i])
test_list

In [ ]:
prediction.head()

In [ ]:
m = tf.keras.metrics.BinaryAccuracy()
m.update_state(test_list, new_list)

In [ ]:
model_accuracy=m.result().numpy()*100
model_accuracy